# Lab 11

## Connect to DBMS

In [1]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost")
collection = client['test']['restaurants']

## Exercise 1

### Query all Indian cuisines

In [2]:
collection.find({'cuisine': 'Indian'})

To show data we can use `list()` or `for`. Then, I'm going to use `list()` with limit.

In [3]:
list(collection.find({'cuisine': 'Indian'}))[:5]

[{'_id': ObjectId('626684816bd12ea62c681296'),
  'address': {'building': '320',
   'coord': [-73.977597, 40.779593],
   'street': 'Columbus Avenue',
   'zipcode': '10023'},
  'borough': 'Manhattan',
  'cuisine': 'Indian',
  'grades': [{'date': datetime.datetime(2014, 10, 27, 0, 0),
    'grade': 'A',
    'score': 7},
   {'date': datetime.datetime(2013, 7, 29, 0, 0), 'grade': 'A', 'score': 5},
   {'date': datetime.datetime(2013, 2, 19, 0, 0), 'grade': 'A', 'score': 11},
   {'date': datetime.datetime(2012, 1, 12, 0, 0), 'grade': 'A', 'score': 2}],
  'name': 'Mughlai Restaurant',
  'restaurant_id': '40370243'},
 {'_id': ObjectId('626684816bd12ea62c681302'),
  'address': {'building': '807',
   'coord': [-73.96643449999999, 40.7641697],
   'street': 'Lexington Avenue',
   'zipcode': '10065'},
  'borough': 'Manhattan',
  'cuisine': 'Indian',
  'grades': [{'date': datetime.datetime(2014, 10, 28, 0, 0),
    'grade': 'A',
    'score': 10},
   {'date': datetime.datetime(2014, 7, 9, 0, 0), 'grade'

### Query all Indian and Thai cuisines

In [4]:
list(collection.find({'cuisine': {'$in': ['Indian', 'Thai']}}))[:5]

[{'_id': ObjectId('626684816bd12ea62c681296'),
  'address': {'building': '320',
   'coord': [-73.977597, 40.779593],
   'street': 'Columbus Avenue',
   'zipcode': '10023'},
  'borough': 'Manhattan',
  'cuisine': 'Indian',
  'grades': [{'date': datetime.datetime(2014, 10, 27, 0, 0),
    'grade': 'A',
    'score': 7},
   {'date': datetime.datetime(2013, 7, 29, 0, 0), 'grade': 'A', 'score': 5},
   {'date': datetime.datetime(2013, 2, 19, 0, 0), 'grade': 'A', 'score': 11},
   {'date': datetime.datetime(2012, 1, 12, 0, 0), 'grade': 'A', 'score': 2}],
  'name': 'Mughlai Restaurant',
  'restaurant_id': '40370243'},
 {'_id': ObjectId('626684816bd12ea62c6812e5'),
  'address': {'building': '244',
   'coord': [-73.9865289, 40.7606981],
   'street': 'West   48 Street',
   'zipcode': '10036'},
  'borough': 'Manhattan',
  'cuisine': 'Thai',
  'grades': [{'date': datetime.datetime(2014, 8, 13, 0, 0),
    'grade': 'A',
    'score': 10},
   {'date': datetime.datetime(2014, 1, 14, 0, 0), 'grade': 'B', 's

### Find a restaurant with the following address: 1115 Rogers Avenue, 11226

In [5]:
collection.find_one({'address.street': 'Rogers Avenue', 'address.building': '1115', 'address.zipcode': '11226'})

{'_id': ObjectId('626684816bd12ea62c6823b4'),
 'address': {'building': '1115',
  'coord': [-73.9513659, 40.6426492],
  'street': 'Rogers Avenue',
  'zipcode': '11226'},
 'borough': 'Brooklyn',
 'cuisine': 'Creole',
 'grades': [{'date': datetime.datetime(2014, 12, 22, 0, 0),
   'grade': 'A',
   'score': 10},
  {'date': datetime.datetime(2014, 8, 5, 0, 0), 'grade': 'A', 'score': 9},
  {'date': datetime.datetime(2014, 2, 10, 0, 0), 'grade': 'A', 'score': 10},
  {'date': datetime.datetime(2012, 12, 19, 0, 0), 'grade': 'A', 'score': 11},
  {'date': datetime.datetime(2011, 12, 30, 0, 0), 'grade': 'A', 'score': 10}],
 'name': 'Chez Macoule Restaurant',
 'restaurant_id': '41012277'}

## Exercise 2
Insert into the database following restaurant:
- Address: 1480 2 Avenue, 10075, -73.9557413, 40.7720266
- Borough: Manhattan
- Cuisine: Italian
- Name: Vella
- Id: 41704620
- Grades:
    - A, 11, 01 Oct, 2014

In [6]:
import datetime
data = {
    'address': {'building': '1480', 'coord': [-73.9557413, 40.7720266], 'street': '2 Avenue', 'zipcode': '10075'},
    'borough': 'Manhattan', 'cuisine': 'Italian',
    'grades': [{'date': datetime.datetime(2014, 10, 1, 0, 0), 'grade': 'A', 'score': 10}],
    'name': 'Vella', 'restaurant_id': '41704620'}
result = collection.insert_one(data)

In [7]:
result.inserted_id

ObjectId('626684978ae54c81f24f7bba')

## Exercise 3

### Delete from the database a single Manhattan located restaurant

In [8]:
collection.delete_one({'borough': 'Manhattan'}).raw_result

{'n': 1, 'ok': 1.0}

### Delete from the database all Thai cuisines

In [9]:
collection.delete_many({'cuisine': 'Thai'}).raw_result

{'n': 285, 'ok': 1.0}

## Exercise 4
Query all restaurants on the Rogers Avenue street, for each of them do the following:
- if it has more than one C grades → delete this restaurant
- otherwise, add another C grade to this restaurant

In [10]:
rests_on_street = collection.find({'address.street': 'Rogers Avenue', 'grades.grade': 'C'})

In [11]:
rests_with_cc = []
for e in rests_on_street:
    c_counter = 0
    for g in e['grades']:
        if g['grade'] == 'C':
            c_counter += 1
    if c_counter > 1:
        collection.delete_one(e)
    else:
        collection.update_one(e, {'$push': {'grades': {'date': datetime.datetime(2014, 10, 1, 0, 0), 'grade': 'C', 'score': 10}}})